# Seasonal Vegetation Anomalies

Notebook is only compatible on the `NCI` as it uses Landsat Collection 2

## Background

Understanding how the vegetated landscape responds to longer-term environmental drivers such as the El Nino Southern Oscillation (ENSO) or climate change, requires the calculation of seasonal anomalies. Seasonal anomalies subtract the long-term seasonal mean from a time-series, thus removing seasonal variability and highlighting change related to longer-term drivers. 

## Description

This notebook will calculate the seasonal anomaly for any given season and year. The long-term seasonal climatologies for the vegetation indices, `MSAVI`, and `NDVI`, have been pre-calculated and are stored on disk. Given an AOI, season, and year, the script will calculate the seasonal mean for one of these indices and subtract the seasonal mean from the long-term climatology, resulting in a map of vegetation anomalies for your AOI.  Optionally, the script will output a geotiff on the result. 

## Technical details

* **Products used:** ls5_nbart_albers, ls7_nbart_albers, ls8_nbart_albers
* **Analyses used:** NDVI, MSAVI, seasonal anomalies

## Getting Started

To run this analysis, go to the `Analysis Parameters` section and enter the relevant details, then run all the cells in the notebook. If running the analysis multiple times, only run the `Set up dask cluster` and `import libraries` cells once.

## Import libraries

In [ ]:
import xarray as xr
from datacube.helpers import write_geotiff
import matplotlib.pyplot as plt
from dask.distributed import Client
import sys
sys.path.append('src')
from anomalies import calculate_anomalies
from anomalies import load_landsat
%load_ext autoreload
%autoreload 2

### Set up local dask cluster

Dask will create a local cluster of cpus for running this analysis in parallel. If you'd like to see what the dask cluster is doing, click on the hyperlink that prints after you run the cell and you can watch the cluster run.

In [ ]:
client = Client(n_workers=6, threads_per_worker=1, memory_limit='4GB')
client

## Analysis Parameters

The following cell sets the parameters, which define the area of interest and the season to conduct the analysis over. The parameters are:

* `veg_index`: The vegetation indice to use for the analysis, either `'msavi'` or `'ndvi'`.
* `from_shape`: If providing a shapefile to the define the area of interest, set this parameter to `True`, otherwise set to `False`.
* `shp_fpath`: If you set `from_shape` to True, provide a filepath to the shapefile
* `lat`, `lon`: If not using a shapefile to define the AOI, then use a latitide and longitude point. This point will be the centre point on your AOI
* `buffer`: The length, in decimal degrees, either side of the `lat` and `lon` point that will define the AOI box. Keep this below `0.5` to avoid long load times
* `year`: The year of interest, e.g. `'2018'`
* `season`:  The season of interest, must be one of `'DJF'`, `'MAM'`, `'JJA'`, or `'SON'`

In [ ]:
veg_index='msavi'
from_shape = False
shp_fpath = "data/griffith_MSAVI_test.shp"
lat, lon = -34.347, 143.832
buffer = 0.25
year = '2017'
season = 'DJF'
chunk_size = 500

## Calculate the anomaly for the AOI

This may take a couple of minutes if the AOI is large

In [ ]:
anomalies = calculate_anomalies(veg_index=veg_index,
                                from_shape=from_shape,
                                shp_fpath=shp_fpath,
                                lat=lat,
                                lon=lon,
                                buffer=buffer,
                                year=year,
                                season=season,
                                chunk_size=chunk_size)

anomalies = anomalies.compute()

## Plot the result

In [ ]:
if veg_index == 'msavi':
    anomalies.msavi_anomalies.plot(figsize=(10,10), vmin=-0.25, vmax=0.25, cmap='BrBG')
if veg_index == 'ndvi':
    anomalies.ndvi_anomalies.plot(figsize=(10,10), vmin=-0.25, vmax=0.25, cmap='BrBG')
plt.title(season+ " " +year)
plt.show()

## Export geotiff

In [ ]:
# Write geotiff to a location
write_geotiff(veg_index+"_"+year+"_"+season+ '_anomalies.tif', anomalies)